In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import pandas as pd
import pickle
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

# Leitura dos dados de treino
df_treino_rf = pd.read_csv('./data/Train.csv')

# Inicialização do CountVectorizer
cv_rf = CountVectorizer(lowercase=True, stop_words='english', ngram_range=(1, 1))

# Transformação dos dados de treino
X_treino_cv_rf = cv_rf.fit_transform(df_treino_rf['text'])

# Inicialização e treinamento do modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_treino_cv_rf, df_treino_rf['label'])

# Definição dos hiperparâmetros
parameters = {
    "n_estimators": [100, 200, 300],  # Número de árvores na floresta
    "max_depth": [2, 4, 6],  # Profundidade máxima das árvores
}

# Procura os melhores hiperparâmetros
grid_search = GridSearchCV(
    rf_model,
    parameters,
    n_jobs=-1,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=5),
)
grid_search.fit(X_treino_cv_rf, df_treino_rf['label'])
print("Todos os parâmetros do melhor modelo:", grid_search.best_estimator_.get_params())

# Função para carregar e pré-processar os dados IMDB
def load_and_preprocess_IMDB(filename, nrows=None):
    """ load the IMDB data and preprocess it:
            - remove html tags
            - remove ponctuation
            - convert to lower case
            - remove stop words
            - remove numbers
            - remove extra spaces
            - replave words with their root form (stem)
            - replace words with their lemma
        :param filename: 'train' or 'test'
        :param nrows: number of rows to read
        :return: df
    """

    # read the data
    df = pd.read_csv(filename, nrows=nrows)

    # keep a copy of the original text
    df['original_text'] = df['text']

    # remove the html tags
    df['text'] = df['text'].str.replace('<br />', ' ')

    # remove the punctuation and '_' characters
    df['text'] = df['text'].str.replace('[^\w\s]', ' ', regex=True)
    df['text'] = df['text'].str.replace('_', ' ', regex=False)

    # convert to lower case
    df['text'] = df['text'].str.lower()

    # remove the stop words
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    # remove the numbers - test on https://regexr.com
    df['text'] = df['text'].str.replace('\d+', '', regex=True)

    # remove the extra spaces - test on https://regexr.com
    df['text'] = df['text'].str.replace(' +', ' ', regex=True)

    # replace the words with their root form
    from nltk.stem import SnowballStemmer
    stemmer = SnowballStemmer('english')
    df['text'] = df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

    # replace the words with their lemma
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

    return df
# Leitura e pré-processamento dos dados de teste
df_teste_rf = load_and_preprocess_IMDB('./data/Test.csv')  # Ajuste o caminho aqui
count_vectors_teste_rf = cv_rf.transform(df_teste_rf['text'])
bow_teste_rf = pd.DataFrame(count_vectors_teste_rf.toarray(), columns=cv_rf.get_feature_names_out())

# Avaliação do modelo nos dados de teste
acuracia_rf = rf_model.score(bow_teste_rf, df_teste_rf['label'])
print(f"Acurácia do Random Forest nos dados de teste: {acuracia_rf}")

# Salvando o modelo e o vetorizador no disco
filename_rf_model = 'rf_model.sav'
pickle.dump(rf_model, open(filename_rf_model, 'wb'))

filename_vectorizer_rf = 'rf_count_vectorizer.sav'
pickle.dump(cv_rf, open(filename_vectorizer_rf, 'wb'))


Todos os parâmetros do melhor modelo: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


C:\Users\Miguel\.conda\envs\aulas\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Acurácia do Random Forest nos dados de teste: 0.8094
